# Pyspark

In [ ]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, year
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName("CustomerPurchaseAnalysis")\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.901") \
    .config("spark.hadoop.fs.s3a.access.key", "ACCESS_KEY")\
    .config("spark.hadoop.fs.s3a.secret.key", "SECRET_KEY")\
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")\
    .config("spark.hadoop.fs.s3a.path.style.access", "true")\
    .config("spark.hadoop.fs.s3a.metastore.metrics.enabled", "false") \
    .config("spark.hadoop.io.native.lib.available", "false")\
    .config("spark.executor.memory", "4g")\
    .config("spark.driver.memory", "4g")\
    .config("spark.hadoop.fs.s3a.region", "ap-south-1") \
    .getOrCreate()

**OS Setup**

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = 'ACCESS_KEY'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'SECRET_KEY'
os.environ['AWS_DEFAULT_REGION'] = 'ap-south-1'  # e.g., 'us-east-1'

**Boto Setup**

In [ ]:
pip install boto3

Boto

In [ ]:
import boto3

# Create a session with your credentials
session = boto3.Session(
    aws_access_key_id='ACCESS_KEY',
    aws_secret_access_key='SECRET_KEY',
    region_name='ap-south-1'  # e.g., 'us-east-1'
)

dynamodb = session.resource('dynamodb')
table = dynamodb.Table('customers')

In [7]:
# Load data from s3
trans_input = "s3a://this-is-my-bucket007/transactions.csv"
trans_df = spark.read.csv(trans_input, header=True, inferSchema=True)


In [8]:
# Load customer data from DynamoDB
response = table.scan()
cust_data = response['Items']

In [9]:
# Convert the DynamoDB items to DataFrame
cust_df = spark.createDataFrame(cust_data)

In [10]:
# Display the initial rows
print("Customer Data from DynamoDB:")
cust_df.show()
print("Transaction Data from S3:")
trans_df.show()

Customer Data from DynamoDB:
+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|       C125|        manoj|
|       C126|          Leo|
|       C123|         John|
|       C124|        brock|
+-----------+-------------+

Transaction Data from S3:
+--------------+-----------+----------------+------+----------------+
|transaction_id|customer_id|transaction_date|amount|product_category|
+--------------+-----------+----------------+------+----------------+
|          T001|       C123|      2024-01-15|  1200|     Electronics|
|          T002|       C124|      2024-01-16|   800|        Clothing|
|          T003|       C125|      2024-01-17|  1500| Home Appliances|
|          T004|       C123|      2024-01-18|  2000|     Electronics|
|          T005|       C126|      2024-01-19|   500|           Books|
|          T006|       C127|      2024-01-20|   300|        Clothing|
|          T007|       C123|      2024-01-21|  4000|       Furniture|
|          T008|    

In [12]:
from pyspark.sql.functions import max

In [13]:
# Calculate last purchase date for each customer
last_purchase_df = (
    trans_df.groupBy("customer_id").agg(max("transaction_date").alias("last_purchase_date"))
)
print("Last purchase date for each customer")
last_purchase_df.show()

Last purchase date for each customer
+-----------+------------------+
|customer_id|last_purchase_date|
+-----------+------------------+
|       C128|        2024-01-24|
|       C123|        2024-01-21|
|       C126|        2024-01-19|
|       C124|        2024-01-23|
|       C125|        2024-01-22|
|       C127|        2024-01-20|
+-----------+------------------+



In [14]:
# Join with customer data to get a complete view
cust_act_df = cust_df.join(last_purchase_df, on="customer_id", how="inner")


In [15]:
from datetime import timedelta

In [16]:
# Flag customers as inactive if last purchase was over six months ago
six_months_ago = datetime.now() - timedelta(days=180)
six_months_ago_str = six_months_ago.strftime("%Y-%m-%d")

In [17]:
from pyspark.sql.functions import datediff, current_date

In [18]:
inactive_cust_df = cust_act_df.withColumn(
    "inactive",(datediff(current_date(), col("last_purchase_date")) > 180)
).select("customer_id", "inactive")

print("customers status")
inactive_cust_df.show()


customers status
+-----------+--------+
|customer_id|inactive|
+-----------+--------+
|       C125|    true|
|       C126|    true|
|       C123|    true|
|       C124|    true|
+-----------+--------+



In [19]:
# Filter out inactive customers
inactive_customers = inactive_cust_df.filter(col("inactive") == True).collect()
active_customers = inactive_cust_df.filter(col("inactive") == False).collect()


In [20]:
# Update status in DynamoDB
for row in inactive_customers:
    table.update_item(
        Key={"customer_id": row["customer_id"]},
        UpdateExpression="SET inactive = :inactive",
        ExpressionAttributeValues={":inactive": {"BOOL": True}}
    )

In [21]:
# For customers who are active, you can update status as 'active' similarly
for row in active_customers:
    table.update_item(
        Key={"customer_id": row["customer_id"]},
        UpdateExpression="SET status = :status",
        ExpressionAttributeValues={":status": {"S": "active"}}
    )

In [22]:
print("Customer status updated in DynamoDB.")

Customer status updated in DynamoDB.


In [23]:
spark.stop()